In [ ]:
# imports
import os
import sys
import cv2
import numpy as np
import subprocess
import pyexif
from pathlib import Path

sys.path.append('..')

from utils.log_utils import log
from utils.multiview_utils import Camera
from utils.metadata_utils import get_cam_names
from configs.arguments import get_config_dict
from utils.io_utils import write_json

In [ ]:
# Load cameras
try:
    config = get_config_dict()
except:
    log.warning("No config file found. Using default values.")
    config = {}
    
data_root = Path(config.get('main', {}).get('data_root', '/root/data'))
omni_tag = config.get('calibration', {}).get('omni_tag', '360')

# set up paths
env_footage = data_root / 'raw_data' / 'footage'
opensfm_data =  data_root / '0-calibration' / 'opensfm'
opensfm_images = opensfm_data / 'images'
extrinsics_path = data_root / '0-calibration' / 'extrinsics'

cams = get_cam_names(env_footage)
camera_models_overrides_dict = {}
log.info(f"Found {(cams)} in environment footage.")
len_360 = 0
sample_images = []
for cam in cams:
    camera = Camera(cam, newest=False)
    if camera.model == omni_tag:
        omni_cam = camera
        len_360 = len(camera)
    sample_images.append(camera.extract([10])[0])



In [ ]:
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display
import math

# Assuming you have a list of lists of images, where each sublist corresponds to a camera
# images = [...]  # replace this with your actual list of images

# Calculate the number of rows and columns for the subplot grid
num_cameras = len(sample_images)
grid_size = math.ceil(math.sqrt(num_cameras))

# A dictionary to store selected frames for each camera and a list to store toggle states
selected_frames = {f'cam{i+1}': [] for i in range(len(sample_images))}
toggle_states = [True] * num_cameras  # Initially all cameras are toggled on

# Function to update the image for the first camera
def update_first_camera(change):
    current_frame = change['new']
    axarr[0].imshow(omni_cam.extract([current_frame])[0])
    plt.draw()

# Function to handle clicks on camera images
def on_click(event, cam_index, frame_index):
    if event.inaxes in axarr[cam_index].get_children():
        if cam_index == 0:
            # Toggle state for the first camera does not affect frame selection
            return
        if toggle_states[cam_index]:
            # Camera is on, add the frame index
            selected_frames[f'cam{cam_index+1}'].append(frame_index)
        else:
            # Toggle the state of the camera
            toggle_states[cam_index] = not toggle_states[cam_index]
            axarr[cam_index].spines['top'].set_color('green' if toggle_states[cam_index] else 'red')
            axarr[cam_index].spines['bottom'].set_color('green' if toggle_states[cam_index] else 'red')
            axarr[cam_index].spines['left'].set_color('green' if toggle_states[cam_index] else 'red')
            axarr[cam_index].spines['right'].set_color('green' if toggle_states[cam_index] else 'red')
            plt.draw()

# Create figure and axes for the images
fig, axarr = plt.subplots(grid_size, grid_size, figsize=(15, 15))
axarr = axarr.flatten()

# Display initial images and setup event handlers
for i in range(num_cameras):
    axarr[i].imshow(sample_images[i])
    axarr[i].axis('off')
    axarr[i].spines['top'].set_color('green' if toggle_states[i] else 'red')
    axarr[i].spines['bottom'].set_color('green' if toggle_states[i] else 'red')
    axarr[i].spines['left'].set_color('green' if toggle_states[i] else 'red')
    axarr[i].spines['right'].set_color('green' if toggle_states[i] else 'red')
    axarr[i].figure.canvas.mpl_connect('button_press_event', lambda event, cam_index=i, frame_index=j: on_click(event, cam_index, frame_index))

# Hide unused subplots
for i in range(num_cameras, grid_size**2):
    axarr[i].axis('off')

# Create a slider for the first camera
slider = widgets.IntSlider(min=0, max=len_360-1, step=1, value=0)
slider.observe(update_first_camera, names='value')

# Display the slider
display(slider)
